In [67]:
import argparse
import sys
import json
from tqdm import tqdm_notebook
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from fun import *

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [165]:
def Get_Artist_Id(artist_name):
   
    """
    Renvoie le spotify artist uri à partir du nom d'un artiste
    """
   
    artist_uri = sp.search(artist_name)['tracks']['items'][0]['artists'][0]['uri']
    artist_str = sp.search(artist_name)['tracks']['items'][0]['artists'][0]["name"]
   
    return artist_uri,artist_str

In [166]:
def Get_Potential_Tracks(artist_uri):
   
    """
    Renvoie tout les tracks succeptibles de provenir d'un artiste
    On rajoute un item albumid
    """
   
    n = 0
    tracks_objects = []
   
    while True:
        
        print("Exploring pages : ",str(n))
        
        albums_objects = sp.artist_albums(artist_uri,limit = 50,offset = n)
        cdt = len(sp.artist_albums(artist_uri,limit=50, offset = n)['items']) > 0
        if not cdt:
            break
       
        albums_ids = [(album["uri"],album["name"],album["album_type"]) for album in albums_objects["items"]]
        
        for albumid,albumname,albumtype in albums_ids:
            
            temptracks = sp.album_tracks(albumid)["items"]
            
            for track in temptracks:
                track["album"] = {"uri":albumid,"name":albumname,"type":albumtype}
            tracks_objects = tracks_objects + temptracks
           
        n = n +50

    return tracks_objects

In [184]:
def Clean_Potential_Tracks(tracks_objects,artist_uri,artist_str,verbose=True):
   
    """
    A partir d'une liste de tracks objects,
    On renvoie les tracks dont l'auteur est l'artiste
    """
   
    cleaned_tracks_objects = []
    for idtrackobj,trackobj in enumerate(tracks_objects):
        
        # TO PRINT
        if verbose:
            toprint = "\r" + str(idtrackobj + 1) + "/" + str(len(tracks_objects))
            print(toprint,end="",flush=True)
        
        track_artists_ids = [(item["uri"],item["name"]) for item in trackobj["artists"]]
        #print(track_artists_ids)
        #track_artists_ids = [item["uri"] for item in trackobj["artists"]]

        if artist_uri in [item[0] for item in track_artists_ids]:
            trackuri = trackobj["uri"]
            audiofeatures= sp.audio_features(trackuri)[0]
            featuring_ids = [item for item in track_artists_ids if item[0] != artist_uri]
            #featuring_ids = [item for item in track_artists_ids if item != artist_uri]
            track_informations = {"artist":artist_uri,
                                  "artist_str" : artist_str,
                                  "featuring":featuring_ids,
                                  "piste":trackobj["track_number"],
                                  "album":trackobj["album"],
                                  "duration":trackobj["duration_ms"],
                                  "name":trackobj["name"],
                                  "track":trackuri,
                                  "features":audiofeatures}
           
            cleaned_tracks_objects.append(track_informations)
   
    return cleaned_tracks_objects

In [185]:
def Retrieve_Artist_Discography(artist_name):
   
    """
    Renvoie un JSON qui représente la discographie d'un artiste
    """
   
    artist_uri = Get_Artist_Id(artist_name)
    tracks_objects = Get_Potential_Tracks(artist_uri)
    discography = Clean_Potential_Tracks(tracks_objects,artist_uri)
   
    return discography

In [186]:
artist_name = "corneille"
artist_uri,artist_str = Get_Artist_Id(artist_name)

In [187]:
tracks_objects = Get_Potential_Tracks(artist_uri)

pages :  0
pages :  50
pages :  100
pages :  150


In [188]:
discography = Clean_Potential_Tracks(tracks_objects,artist_uri,artist_str)

1994/1994

### DRAFT

In [88]:
artist = "corneille"

In [89]:
artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']

In [112]:
artist_obj = sp.search("corneille")

In [91]:
al = sp.artist_albums(artist_uri,limit = 50)

In [92]:
al = sp.artist_albums(artist_uri,limit = 50)
alid = al["items"][49]["uri"]
alobj = sp.album_tracks(alid)

In [119]:
al["items"][48]

{'album_group': 'appears_on',
 'album_type': 'compilation',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
   'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
   'id': '0LyfQWJT6nXafLPZqxe9Of',
   'name': 'Various Artists',
   'type': 'artist',
   'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
 'available_markets': ['AD', 'FR', 'MC'],
 'external_urls': {'spotify': 'https://open.spotify.com/album/0TBI8FUxqDeQ6TkhIURczx'},
 'href': 'https://api.spotify.com/v1/albums/0TBI8FUxqDeQ6TkhIURczx',
 'id': '0TBI8FUxqDeQ6TkhIURczx',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273dc2fa557d567636b7ac59d3b',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02dc2fa557d567636b7ac59d3b',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851dc2fa557d567636b7ac59d3b',
   'width': 64}],
 'name': '50 Hits Summer 2020',
 'release_da

In [93]:
alobj["items"][1]

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5YFS41yoX0YuFY39fq21oN'},
   'href': 'https://api.spotify.com/v1/artists/5YFS41yoX0YuFY39fq21oN',
   'id': '5YFS41yoX0YuFY39fq21oN',
   'name': 'Bob Sinclar',
   'type': 'artist',
   'uri': 'spotify:artist:5YFS41yoX0YuFY39fq21oN'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/2cSQyZg5t274Y3Nv0uuwjd'},
   'href': 'https://api.spotify.com/v1/artists/2cSQyZg5t274Y3Nv0uuwjd',
   'id': '2cSQyZg5t274Y3Nv0uuwjd',
   'name': 'Gary Pine',
   'type': 'artist',
   'uri': 'spotify:artist:2cSQyZg5t274Y3Nv0uuwjd'}],
 'available_markets': ['AD', 'FR', 'MC'],
 'disc_number': 1,
 'duration_ms': 205489,
 'explicit': False,
 'external_urls': {'spotify': 'https://open.spotify.com/track/0oxtni0bToH7ZdeUqvueAe'},
 'href': 'https://api.spotify.com/v1/tracks/0oxtni0bToH7ZdeUqvueAe',
 'id': '0oxtni0bToH7ZdeUqvueAe',
 'is_local': False,
 'name': 'Love Generation (feat. Gary Pine)',
 'preview_url': 'https://p.scdn.co/

In [40]:
trackobj = sp.track(trackuri)

In [ ]:
spotify:track:4QAbMOJ4O0cckSnh6FS5Hv

In [44]:
trackobj

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2GBDOaCl73BVUs2RerIme7'},
    'href': 'https://api.spotify.com/v1/artists/2GBDOaCl73BVUs2RerIme7',
    'id': '2GBDOaCl73BVUs2RerIme7',
    'name': 'Corneille',
    'type': 'artist',
    'uri': 'spotify:artist:2GBDOaCl73BVUs2RerIme7'}],
  'available_markets': ['AD',
   'AL',
   'AT',
   'BA',
   'BE',
   'BG',
   'BY',
   'CH',
   'CZ',
   'DE',
   'DK',
   'DZ',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'HR',
   'HU',
   'IE',
   'IS',
   'IT',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MD',
   'ME',
   'MK',
   'MT',
   'NL',
   'NO',
   'PL',
   'PT',
   'RO',
   'RS',
   'RU',
   'SE',
   'SI',
   'SK',
   'UA',
   'XK'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0gwGnBt9CbWuZTN9flLZ7i'},
  'href': 'https://api.spotify.com/v1/albums/0gwGnBt9CbWuZTN9flLZ7i',
  'id': '0gwGnBt9CbWuZTN9flLZ7i',
  'images': [{'height': 

In [63]:
x = "/labels/1119"

In [65]:
import os.path

In [66]:
os.path.isfile("where.sh")

True

In [59]:
class Crawler:
    
    def __init__(self,graine):
        self.bac = graine
        self.artists = graine
        self.runs = 0
        
    def Run():
        self.runs = self.runs + 1
        self.bac = 
            
            

In [60]:
mycrawler = Crawler(["jeff mills"])

In [62]:
mycrawler.graine

['jeff mills']